In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def process_image(BG: np.ndarray, I1: np.ndarray, I2: np.ndarray) -> np.ndarray:
    """
    Process images by removing green screen background and compositing them onto a background image.
    
    Args:
        BG (np.ndarray): Background image
        I1 (np.ndarray): First foreground image (male)
        I2 (np.ndarray): Second foreground image (female)
    
    Returns:
        np.ndarray: Composited image with both foregrounds placed on the background
    """
    b , g , r = cv2.split(I1)
    b1 , g1 , r1 = cv2.split(I2)

    hsv = cv2.cvtColor(I1, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    hsv1 = cv2.cvtColor(I2, cv2.COLOR_BGR2HSV)
    h1,s1,v1 = cv2.split(hsv1)

    for channel, label in zip([b,g,r,h,s,v], ['Blue','Green','Red','Hue','Saturation','Value']):
        mean = np.mean(channel)
        std = np.std(channel)
        print(f"Image 1 {label} - Mean: {mean:.2f}, Std: {std:.2f}")
    
    for channel, label in zip([b1,g1,r1,h1,s1,v1], ['Blue','Green','Red','Hue','Saturation','Value']):
        mean = np.mean(channel)
        std = np.std(channel)
        print(f"Image 2 {label} - Mean: {mean:.2f}, Std: {std:.2f}")

    plt.figure(figsize=(15,5))
        
    # BGR histogram
    plt.subplot(121)
    plt.hist(b.ravel(), bins=256, color='b', alpha=0.5)
    plt.hist(g.ravel(), bins=256, color='g', alpha=0.5) 
    plt.hist(r.ravel(), bins=256, color='r', alpha=0.5)
   
    
    # HSV histogram
    plt.subplot(122)
    plt.hist(h.ravel(), bins=180, color='r', alpha=0.5)
    plt.hist(s.ravel(), bins=256, color='g', alpha=0.5)
    plt.hist(v.ravel(), bins=256, color='b', alpha=0.5)

        
    plt.show()
    

    
    I1_hsv = I1.copy()
    I1_hsv = cv2.cvtColor(I1_hsv, cv2.COLOR_BGR2HSV)
    I2_hsv = I2.copy()
    I2_hsv = cv2.cvtColor(I2_hsv, cv2.COLOR_BGR2HSV)

    lower_bound = np.array([30, 50, 50])
    upper_bound = np.array([100, 200, 255])


    fig, ax = plt.subplots(2, 2, figsize=(10, 5))
    # Show original images and their HSV representations
    ax[0, 0].imshow(cv2.cvtColor(I1, cv2.COLOR_BGR2RGB))
    ax[0, 0].set_title('Original')
    ax[0, 0].axis('off')
    ax[0, 1].imshow(I1_hsv)
    ax[0, 1].set_title('HSV')
    ax[0, 1].axis('off')

    ax[1, 0].imshow(cv2.cvtColor(I2, cv2.COLOR_BGR2RGB))
    ax[1, 0].set_title('Original')
    ax[1, 0].axis('off')
    ax[1, 1].imshow(I2_hsv)
    ax[1, 1].set_title('HSV')
    ax[1, 1].axis('off')

    # Create initial masks
    mask1 = cv2.inRange(I1_hsv, lower_bound, upper_bound)
    mask2 = cv2.inRange(I2_hsv, lower_bound, upper_bound)
    
    # Clean up masks with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    
    # First close to fill gaps
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel)
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel)
    
    # Then open to remove noise
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel)
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_OPEN, kernel)
    
    # Extract foreground
    fg1 = cv2.bitwise_and(I1, I1, mask=mask1)
    fg2 = cv2.bitwise_and(I2, I2, mask=mask2)
    
    # Create inverse masks for the background
    mask1_inv = cv2.bitwise_not(mask1)
    mask2_inv = cv2.bitwise_not(mask2)

   # make sure type is uint8
    mask1_inv = np.uint8(mask1_inv)
    mask2_inv = np.uint8(mask2_inv)
    

    
    
    # Get background regions
    bg1 = cv2.bitwise_and(BG, BG, mask=mask1_inv)
    bg2 = cv2.bitwise_and(BG, BG, mask=mask2_inv)

   

    
    # Combine foreground and background
    result = cv2.add(bg1, fg1)
    result = cv2.add(result, fg2)



    
    plt.show()

    
    
    return result
    

   
    

# Please DO NOT change any of the code below. All modifications to this template should
# occur inside the **process_image** function


BG = cv2.imread('./background.jpg')
I1 = cv2.imread('./male.jpg')
I2 = cv2.imread('./female.jpg')

merged = process_image(BG, I1, I2)

plt.imshow(merged)
plt.show()


: 